# Descripción

Este notebook contiene los pasos requeridos para asignar las combinaciones, previamente asignadas, profesor-curso a un salón de acuerdo a los requerimientos institucionales.

In [1]:
import json


# READ DATA

## first stage model output

In [2]:
# basic parameters
DAY_LENGTH=16

# Horizonte de planificación
HORIZON=5

#  Días de la semana
DAYS=['Lunes','Martes','Miercoles','Jueves','Viernes']


In [3]:
# Este diccionario contiene un mapa de los slots de tiempo a los días y horas

actual_time={}
for d in range(HORIZON):
    for h in range(DAY_LENGTH):
        actual_time[d*16+h]={"slot":DAYS[d]+' '+str(h+6)+':00', "day":d, "hour":h,'slot_abr':DAYS[d][0]+str(h)}

In [4]:
# read model_output.json
allocations=json.load(open('data/model_output.json','r'))
#allocations

In [5]:

groups=allocations['courses']
teachers=allocations['teachers']
pairs=allocations['ensembles']
teachers=set([int(t) for t in teachers])
groups=set([g for g in groups])
pairs=[(int(p[0]) ,p[1]) for p in pairs]


In [6]:
courses_by_teacher={}
for p in pairs:
    if p[0] not in courses_by_teacher:
        courses_by_teacher[p[0]]=[p[1]]
    else:
        courses_by_teacher[p[0]].append(p[1])


## Initial data load

In [7]:
# read data.json
horas_curso=json.load(open('data/horas_curso.json','r'))
grupo_requerimiento=json.load(open('data/grupo_requerimiento.json','r'))
capacidad_salones=json.load(open('data/capacidad_sala.json','r'))
ubicacion_semestral=json.load(open('data/ubicacion_semestral.json','r'))

# Model construction

In [8]:
import pulp as plp
import numpy as np

I=set(teachers)
J=set(groups)
K=set(pairs)
C=set(capacidad_salones.keys())



In [20]:
from itertools import product

# courses that are the same
same_course={}
# courses that are taught by the same teacher
same_teacher={}

# delta semestres
delta_semestre={}
for j1,j2 in product(J,J):
    same_course[(j1,j2)]=0
    same_teacher[(j1,j2)]=0
    if j1[:9]==j2[:9]:
        same_course[(j1,j2)]=1
    delta_semestre[(j1,j2)]=abs(ubicacion_semestral[j1[:9]]["Semestre"]-ubicacion_semestral[j2[:9]]["Semestre"])

for p in pairs:
    for p2 in pairs:
        if p[0]==p2[0]:
            same_teacher[(p[1],p2[1])]=1



In [21]:
delta_semestre

{('5555-0010-0', '5555-0010-0'): 0,
 ('5555-0010-0', '9935-0012-0'): 4,
 ('5555-0010-0', '6611-0004-0'): 6,
 ('5555-0010-0', '6612-0001-0'): 3,
 ('5555-0010-0', '2027-0059-1'): 7,
 ('5555-0010-0', '5559-0014-0'): 5,
 ('5555-0010-0', '8821-0002-0'): 1,
 ('5555-0010-0', '8827-0120-0'): 0,
 ('5555-0010-0', '9937-0013-0'): 2,
 ('5555-0010-0', '9937-0063-0'): 7,
 ('5555-0010-0', '9937-0014-0'): 0,
 ('5555-0010-0', '9937-0018-0'): 1,
 ('5555-0010-0', '9937-0011-0'): 4,
 ('5555-0010-0', '8830-0012-0'): 1,
 ('5555-0010-0', '8830-0008-0'): 3,
 ('5555-0010-0', '9937-0062-0'): 7,
 ('5555-0010-0', '9935-0031-0'): 6,
 ('5555-0010-0', '8833-0083-0'): 2,
 ('5555-0010-0', '8827-0008-0'): 2,
 ('5555-0010-0', '6069-0078-0'): 1,
 ('5555-0010-0', '6069-0012-0'): 1,
 ('5555-0010-0', '5566-0009-0'): 1,
 ('5555-0010-0', 'CEEM-0014-0'): 1,
 ('5555-0010-0', '8821-0010-0'): 0,
 ('5555-0010-0', '8833-0021-0'): 3,
 ('5555-0010-0', '8831-0028-0'): 5,
 ('5555-0010-0', '6069-0154-0'): 1,
 ('5555-0010-0', '8828-0028-

In [22]:
# course and time slot compatibility
time_compatibility={}
for c,j in product(C,J):
    for s in actual_time.keys():
        time_compatibility[(c,j,s)]=0
        if actual_time[s]['hour'] + horas_curso[j[:9]]['horas']<=DAY_LENGTH:
            time_compatibility[(c,j,s)]=1

In [23]:
valid_keys=[]
for c,j,s in product(C,J,actual_time.keys()):
    if time_compatibility[(c,j,s)]==1:
        valid_keys.append((c,j,s))



In [24]:
valid_keys[:10]

[('PALB01-214', '5555-0010-0', 0),
 ('PALB01-214', '5555-0010-0', 1),
 ('PALB01-214', '5555-0010-0', 2),
 ('PALB01-214', '5555-0010-0', 3),
 ('PALB01-214', '5555-0010-0', 4),
 ('PALB01-214', '5555-0010-0', 5),
 ('PALB01-214', '5555-0010-0', 6),
 ('PALB01-214', '5555-0010-0', 7),
 ('PALB01-214', '5555-0010-0', 8),
 ('PALB01-214', '5555-0010-0', 9)]

In [26]:
# the course starts at time slot s
y=plp.LpVariable.dicts('y',valid_keys,0,1,plp.LpBinary)

# the course is taught in classroom c at time slot s
x=plp.LpVariable.dicts('x',[(c,j,s) for c in C for j in J for s in actual_time.keys()],0,1,plp.LpBinary)

# la clase a y la clase b se solapan
z=plp.LpVariable.dicts('z',[(j1,j2) for j1,j2 in product(J,J) if j1!=j2],0,1,plp.LpBinary)


In [27]:
model=plp.LpProblem('Asignacion de salones',plp.LpMaximize)

# funcion objetivo:
# maximizar la suma de los deltas semestrales de los cursos que se imparten
model+=plp.lpSum([z[(j1,j2)]*delta_semestre[(j1,j2)] for j1,j2 in product(J,J) if j1!=j2])
model

Asignacion_de_salones:
MAXIMIZE
7*z_('2027_0059_0',_'5555_0010_0') + 2*z_('2027_0059_0',_'5559_0014_0') + 8*z_('2027_0059_0',_'5566_0009_0') + 4*z_('2027_0059_0',_'5569_0011_0') + 4*z_('2027_0059_0',_'5569_0011_1') + 4*z_('2027_0059_0',_'5569_0011_2') + 5*z_('2027_0059_0',_'6069_0011_0') + 8*z_('2027_0059_0',_'6069_0012_0') + 3*z_('2027_0059_0',_'6069_0014_0') + 6*z_('2027_0059_0',_'6069_0018_0') + 8*z_('2027_0059_0',_'6069_0078_0') + 6*z_('2027_0059_0',_'6069_0127_0') + 8*z_('2027_0059_0',_'6069_0154_0') + 9*z_('2027_0059_0',_'6069_0175_0') + 7*z_('2027_0059_0',_'6069_2060_0') + 1*z_('2027_0059_0',_'6611_0004_0') + 1*z_('2027_0059_0',_'6611_0004_1') + 5*z_('2027_0059_0',_'6611_0088_0') + 4*z_('2027_0059_0',_'6612_0001_0') + 8*z_('2027_0059_0',_'8821_0002_0') + 7*z_('2027_0059_0',_'8821_0010_0') + 5*z_('2027_0059_0',_'8827_0008_0') + 7*z_('2027_0059_0',_'8827_0120_0') + 4*z_('2027_0059_0',_'8828_0028_0') + 3*z_('2027_0059_0',_'8830_0004_0') + 3*z_('2027_0059_0',_'8830_0005_0') + 5*z_('

In [28]:

# each course starts at one time slot
for j in J:
    model+=plp.lpSum([y[(c,j1,s)] for c,j1,s in valid_keys if j1==j])==1

In [29]:
# if a course start at time slot s in classroom c, it should last for the required time
contador=0
for c,j,s in valid_keys:
    contador+=1
    expr=plp.lpSum([x[(c,j,s1)] for s1 in range(s,s+horas_curso[j[:9]]['horas'])  if (c,j,s1) in valid_keys])>=y[(c,j,s)]*horas_curso[j[:9]]['horas']
    model+=expr
    if contador>10:
        pass


In [46]:
# each course is assined the time it requires
for j in J:
    model+=plp.lpSum([x[(c,j1,s)] for c,j1,s in valid_keys if j1==j])==horas_curso[j[:9]]['horas']

In [48]:
# si dos cursos se imparten a la misma hora en cualquier salon se debe activar la variable z, que lleva cuentas de los solapamientos

for j1,j2 in product(J,J):
    if j1!=j2:
        for c1 ,c2 in product(C,C):
            for s in actual_time.keys():
                model+=z[(j1,j2)]>=x[(c1,j1,s)]+x[(c2,j2,s)]-1

KeyboardInterrupt: 

In [ ]:
solver=plp.GUROBI_CMD()

In [39]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/user/Library/Caches/pypoetry/virtualenvs/timetabling_upb/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/2d0a6ae1821d420486a377988456e12e-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/2d0a6ae1821d420486a377988456e12e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 121 COLUMNS
At line 727340 RHS
At line 727457 BOUNDS
At line 969864 ENDATA
Problem MODEL has 116 rows, 242406 columns and 239100 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.22 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was

1